# Lecture 21

In [1]:
from dwave.system.samplers import DWaveSampler

Let's find out which quantum annealers are accessible to you. 

In [2]:
from dwave.cloud import Client
# If the command below doesn't work, add your token by using this:
# client = Client.from_config(token='add your token here')
client = Client.from_config()
client.get_solvers()

[StructuredSolver(id='DW_2000Q_6'), StructuredSolver(id='Advantage_system4.1')]

 There are two quantum devices available for me. DW_2000Q_6 is a 2000 qubit quantum annealer with low noise and the Chimera graph. 
Advantage_system1.1 is a 5000 quantum annealer with ~3X higher noise, and the Pegasus graph. 
Let's use the 2000 qubit one:

In [3]:
sampler_2000 = DWaveSampler(id='DW_2000Q_6')

Let's find out which qubits are available:

In [4]:
sampler_2000.nodelist

[30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 2

And which couplings are available:

In [11]:
sampler_2000.edgelist

[(30, 31),
 (30, 45),
 (30, 2940),
 (30, 2955),
 (30, 2970),
 (30, 2985),
 (31, 32),
 (31, 46),
 (31, 3120),
 (31, 3135),
 (31, 3150),
 (31, 3165),
 (32, 33),
 (32, 47),
 (32, 3300),
 (32, 3315),
 (32, 3330),
 (32, 3345),
 (33, 34),
 (33, 48),
 (33, 3480),
 (33, 3495),
 (33, 3510),
 (33, 3525),
 (34, 35),
 (34, 49),
 (34, 3660),
 (34, 3675),
 (34, 3690),
 (34, 3705),
 (35, 36),
 (35, 50),
 (35, 3840),
 (35, 3855),
 (35, 3870),
 (35, 3885),
 (36, 37),
 (36, 51),
 (36, 4020),
 (36, 4035),
 (36, 4050),
 (36, 4065),
 (37, 38),
 (37, 52),
 (37, 4200),
 (37, 4215),
 (37, 4230),
 (37, 4245),
 (38, 39),
 (38, 53),
 (38, 4380),
 (38, 4395),
 (38, 4410),
 (38, 4425),
 (39, 40),
 (39, 54),
 (39, 4560),
 (39, 4575),
 (39, 4590),
 (39, 4605),
 (40, 41),
 (40, 55),
 (40, 4740),
 (40, 4755),
 (40, 4770),
 (40, 4785),
 (41, 42),
 (41, 56),
 (41, 4920),
 (41, 4935),
 (41, 4950),
 (41, 4965),
 (42, 43),
 (42, 57),
 (42, 5100),
 (42, 5115),
 (42, 5130),
 (42, 5145),
 (43, 44),
 (43, 58),
 (43, 5280),
 (4

# 1) Implement simple QUBO with two qubits in the Chimera graph: 
$$E(q30,q31)=(q30-q31)^2.$$

Set Q for the minor-embedded problem QUBO

In [5]:
qubit_biases = {(30, 30): 1, (31, 31): 1}
coupler_strengths = {(30, 31): -2}
Q = dict(qubit_biases)
Q.update(coupler_strengths)

In [6]:
response = sampler_2000.sample_qubo(Q, num_reads=100)
for (sample, energy, num) in response.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num)

{30: 0, 31: 0} Energy:  0.0 Occurrences:  58
{30: 1, 31: 1} Energy:  0.0 Occurrences:  42


As you can see, ALL of the time the QPU outputs values with qubits in the same state (00 or 11).

In [7]:
# Let's solve this "problem" using "brute force": Check all possible 2^n solutions:
from dimod.reference.samplers import ExactSolver
sampler_CPU = ExactSolver()

In [9]:
responseBruteForce = sampler_CPU.sample_qubo(Q)
for (sample, energy, num) in responseBruteForce.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num)

{30: 0, 31: 0} Energy:  0.0 Occurrences:  1
{30: 1, 31: 1} Energy:  0.0 Occurrences:  1
{30: 1, 31: 0} Energy:  1.0 Occurrences:  1
{30: 0, 31: 1} Energy:  1.0 Occurrences:  1


## 2) Minor-embedding to implement 3-bit 3-SAT

We will now use "chaining" to lock q30 and q45 in the same state. This will allow the use of 
$(q30, q31, q46, q45)$ to represent 3 qubits coupled to each other with the same coupling strengths. 
Entering the biases and coupler strengths as described in the notes:

In [10]:
qubit_biases = {(30, 30): 0.33, (46, 46): -0.33, (31, 31): -0.33, (45, 45): 0.33}
coupler_strengths = {(30, 31): 0.66, (46, 31): 0.66, (46, 45): 0.66, (30, 45): -1}
Q = dict(qubit_biases)
Q.update(coupler_strengths)

In [11]:
response = sampler_2000.sample_qubo(Q, num_reads=1000)
for (sample, energy, num) in response.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num)

{30: 1, 31: 0, 45: 1, 46: 0} Energy:  -0.33999999999999997 Occurrences:  315
{30: 0, 31: 0, 45: 0, 46: 1} Energy:  -0.33000000000000007 Occurrences:  416
{30: 0, 31: 1, 45: 0, 46: 0} Energy:  -0.32999999999999996 Occurrences:  265
{30: 1, 31: 0, 45: 1, 46: 1} Energy:  -0.009999999999999981 Occurrences:  2
{30: 1, 31: 0, 45: 0, 46: 1} Energy:  -2.7755575615628914e-17 Occurrences:  1
{30: 0, 31: 1, 45: 0, 46: 1} Energy:  -2.7755575615628914e-17 Occurrences:  1


The trick worked out nicely! We got each of the 3 right answers. One of them has slightly less energy and appeared about 1/2 of the time. 

The other two have the same energy and appeared 1/4 of the time each. 


## 3) Using DWave's EmbeddingComposite function to do the minor-embedding for you


Let's implement the 3-SAT example from class:
Four bits z0 z1 z2 z3 z4
Three clauses C(0,1,2), C(1,2,3), C(0,3,4)


In [13]:
from dwave.system.composites import EmbeddingComposite

Set Q for the problem QUBO. 

Note: The linear coefficients are -1 * number of occurrence of z_i in all clauses;
The quadratic coefficients are 2* number of occurrence of pair in all clauses.


In [14]:
linear = {('z0', 'z0'): -2, ('z1', 'z1'): -2, ('z2', 'z2'): -2, ('z3', 'z3'): -2, ('z4', 'z4'): -1}
quadratic = {('z0', 'z1'): 2, ('z0', 'z2'): 2, ('z1', 'z2'): 4, ('z1', 'z3'): 2, ('z2', 'z3'): 2, ('z0', 'z3'): 2, ('z0', 'z4'): 2, ('z3', 'z4'): 2}
Q = dict(linear)
Q.update(quadratic)

Minor-embed and sample 1000 times on a default D-Wave system:

In [15]:
response = EmbeddingComposite(sampler_2000).sample_qubo(Q, num_reads=1000)
for (sample, energy, num_occurrences, aux) in response.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num_occurrences)

{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 1} Energy:  -3.0 Occurrences:  371
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 1} Energy:  -3.0 Occurrences:  522
{'z0': 1, 'z1': 0, 'z2': 0, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  8
{'z0': 0, 'z1': 0, 'z2': 0, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  8
{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  16
{'z0': 1, 'z1': 0, 'z2': 0, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  13
{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  9
{'z0': 1, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  16
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  13
{'z0': 1, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  8
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  14
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 1, 'z4': 1} Energy:  -1.0 Occurrences:  1
{'z0': 0, 'z1': 1, 'z2': 1, 'z3': 0, 'z4': 1} Energy:  -1.0 Occurrences:  1


Are these the correct solutions we agreed in class? :-) Yes! They occurred 371 + 522 = 893 times (89.3% of the time)

Let's try to use their new 5000 qubit device. It has more noise though. 
I will define the sampler with the function Embedding Composite already there (convenient!)

In [16]:
sampler_advantage = EmbeddingComposite(DWaveSampler(id='Advantage_system1.1'))

In [17]:
response_adv = sampler_advantage.sample_qubo(Q, num_reads=1000)
for (sample, energy, num_occurrences, aux) in response_adv.data():
    print(sample, "Energy: ", energy, "Occurrences: ", num_occurrences)

{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 1} Energy:  -3.0 Occurrences:  456
{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 1} Energy:  -3.0 Occurrences:  418
{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  11
{'z0': 0, 'z1': 0, 'z2': 0, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  8
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  16
{'z0': 0, 'z1': 0, 'z2': 1, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  16
{'z0': 1, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  24
{'z0': 1, 'z1': 0, 'z2': 0, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  15
{'z0': 1, 'z1': 0, 'z2': 0, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  12
{'z0': 1, 'z1': 0, 'z2': 1, 'z3': 0, 'z4': 0} Energy:  -2.0 Occurrences:  10
{'z0': 0, 'z1': 1, 'z2': 0, 'z3': 1, 'z4': 0} Energy:  -2.0 Occurrences:  11
{'z0': 0, 'z1': 1, 'z2': 1, 'z3': 0, 'z4': 1} Energy:  -1.0 Occurrences:  1
{'z0': 1, 'z1': 1, 'z2': 0, 'z3': 0, 'z4': 1} Energy:  -1.0 Occurrences:  1


The advantage_system got it right 456 + 418 = 874 times, quite similar to the dwave_2000. 